In [2]:
import os
from itertools import combinations
from scipy.special import comb
from IPython.display import display, HTML

from tools.compare_functions import load_solutions, get_pareto_front, get_styled_table, classify_solutions

import pandas as pd
from tqdm import tqdm 
import numpy as np
import pandas as pd



# Datos

In [3]:
archive = "WorkSpace 1000_50_10"
k=10
m=50

# Definición de la búsqueda exhaustiva

In [ ]:
class BusquedaExhaustiva:
    def __init__(self, archive, k, m):
        """
        Inicializa el algoritmo de búsqueda exhaustiva.
        
        Args:
            archive (str): Nombre base del archivo de distancias (sin .csv).
            k (int): Número de puntos de suministro a seleccionar en cada solución.
            m (int): Número total de puntos de suministro disponibles.
        """
        # Carga la matriz de distancias
        folder_distances = "./data/distances/demand/"
        route_distances = os.path.join(folder_distances, f"{archive}.csv")

        # --- LÍNEA MODIFICADA ---
        # Usamos index_col=0 para que la primera columna del CSV se trate como el índice de filas.
        # Pandas ya interpreta la primera fila como encabezado por defecto.
        self.df_distances_demand = pd.read_csv(route_distances, index_col=0)
        # -------------------------

        # Prepara el archivo de soluciones finales
        self.route_solutions = f"{archive}.csv"
        columnas = ["solution", "f1", "f2", "f3"]
        self.df_solutions = pd.DataFrame(columns=columnas)
        
        # Parámetros del problema
        self.k = k
        self.m = m
        
        # Valida que m sea el número de columnas en el df de distancias
        if self.m != len(self.df_distances_demand.columns):
            raise ValueError(
                f"El valor de m ({self.m}) no coincide con el número de columnas "
                f"({len(self.df_distances_demand.columns)}) en el archivo de distancias después de "
                "establecer la primera columna como índice."
            )

    ####
    # F1: Maximizar la mínima distancia cubierta
    ####
    
    def f1(self, supply_selected):
        """
        Calcula el máximo de las distancias mínimas para cada punto de demanda
        a los puntos de suministro seleccionados, usando operaciones vectorizadas.
        """
        # 1. Selecciona las columnas de la oferta seleccionada.
        # 2. Calcula la distancia mínima para cada fila (axis=1).
        # 3. Devuelve el valor máximo de esas distancias mínimas.
        return self.df_distances_demand.iloc[:, supply_selected].min(axis=1).max()

    ####
    # F2: Equilibrar la carga máxima
    ####

    
    def f2(self, supply_selected):
        asignacion = self.df_distances_demand.iloc[:, supply_selected].idxmin(axis=1)
        maximum = asignacion.value_counts().max()
        return maximum

    ####
    # F3: Equilibrar la carga general
    ####
    
    def f3(self, supply_selected):
        """
        Calcula la diferencia entre el número máximo y mínimo de puntos de demanda
        asignados, asegurando que se consideren todos los puntos de suministro seleccionados.
        """
        # Obtiene los nombres/etiquetas de las columnas seleccionadas
        supply_columns = self.df_distances_demand.columns[supply_selected]
        
        # Determina la asignación de demanda al punto de suministro más cercano
        asignacion = self.df_distances_demand[supply_columns].idxmin(axis=1)
        
        # Cuenta las asignaciones para cada punto de suministro
        counts = asignacion.value_counts()
        
        # Reindexa para incluir todos los puntos de suministro seleccionados,
        # rellenando con 0 los que no recibieron asignaciones.
        all_counts = counts.reindex(supply_columns, fill_value=0)
        
        return all_counts.max() - all_counts.min()

    def _add_solutions(self, solution, f1, f2, f3):
        solution = str(sorted(solution))
        if not self.df_solutions.empty:
            if solution in self.df_solutions["solution"].values:
                print("la solucion ya existe")
                return  # La solución ya existe, no hacer nada

            df_dominado = self.df_solutions.copy()
            df_dominado = df_dominado[df_dominado["f1"] <= f1]
            df_dominado = df_dominado[df_dominado["f2"] <= f2]
            df_dominado = df_dominado[df_dominado["f3"] <= f3]
            if df_dominado.empty:
                # Eliminar soluciones que sean dominadas por la nueva
                self.df_solutions = self.df_solutions[
                    ~(
                        (self.df_solutions["f1"] >= f1)
                        & (self.df_solutions["f2"] >= f2)
                        & (self.df_solutions["f3"] >= f3)
                        & (
                            (self.df_solutions["f1"] > f1)
                            | (self.df_solutions["f2"] > f2)
                            | (self.df_solutions["f3"] > f3)
                        )
                    )
                ]
                # Agregar la nueva solución
                new_solution = pd.DataFrame(
                    [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
                )

                self.df_solutions = pd.concat(
                    [self.df_solutions, new_solution], ignore_index=True
                )
                self.df_solutions.to_csv(self.route_solutions, index=False)
        else:
            self.df_solutions = pd.DataFrame(
                [{"solution": solution, "f1": f1, "f2": f2, "f3": f3}]
            )
            self.df_solutions.to_csv(self.route_solutions, index=False)
        return

    def run(self):
        """
        Ejecuta la búsqueda exhaustiva con una barra de progreso.
        """
        print("Iniciando búsqueda exhaustiva...")
        
        potential_supplies = list(range(self.m))
        
        # Calcular el número total de combinaciones de forma eficiente
        num_combinations = comb(self.m, self.k)
        print(f"Se evaluarán {num_combinations} combinaciones. ¡Esto puede tardar mucho tiempo! ⏳")

        all_combinations = combinations(potential_supplies, self.k)
        
        # Envolver el iterador con tqdm para mostrar la barra de progreso
        pbar = tqdm(all_combinations, total=num_combinations, desc="Procesando Combinaciones", unit=" comb")
        
        for current_solution in pbar:
            solution_list = list(current_solution)
            
            # Calcular los tres objetivos
            f1_value = self.f1(solution_list)
            f2_value = self.f2(solution_list)
            f3_value = self.f3(solution_list)

            print((solution_list, f1_value, f2_value, f3_value))
            
            # Añadir al conjunto de Pareto si corresponde
            self._add_solutions(solution_list, f1_value, f2_value, f3_value)
            
            # Opcional: Actualizar la descripción de la barra de progreso con información en tiempo real
            pbar.set_postfix(pareto_count=len(self.df_solutions))

        print(f"\nBúsqueda completada. Se encontraron {len(self.df_solutions)} soluciones no dominadas. ✅")
        print(f"Los resultados se han guardado en '{self.route_solutions}'.")
        print("\nDataFrame final de soluciones de Pareto:")
        print(self.df_solutions)

In [4]:
class BusquedaExhaustivaOptimizada:
    def __init__(self, archive, k, m):
        """
        Inicializa el algoritmo de búsqueda exhaustiva optimizado.
        
        Args:
            archive (str): Nombre base del archivo de distancias (sin .csv).
            k (int): Número de puntos de suministro a seleccionar.
            m (int): Número total de puntos de suministro disponibles.
        """
        # Carga la matriz de distancias una sola vez
        folder_distances = "./data/distances/demand/"
        route_distances = os.path.join(folder_distances, f"{archive}.csv")
        df_distances_demand = pd.read_csv(route_distances, index_col=0)

        # --- MEJORA 1: Conversión a NumPy ---
        # Usamos un array de NumPy para todos los cálculos numéricos. Es mucho más rápido.
        self.dist_matrix = df_distances_demand.to_numpy()
        
        # Guardamos los nombres de las columnas para el resultado final
        self.supply_point_names = df_distances_demand.columns.to_numpy()
        
        # Parámetros del problema
        self.k = k
        self.m = m
        self.route_solutions = f"{archive}.csv"
        
        if self.m != self.dist_matrix.shape[1]:
            raise ValueError(
                f"El valor de m ({self.m}) no coincide con el número de columnas "
                f"({self.dist_matrix.shape[1]}) en el archivo de distancias."
            )
        
        # --- MEJORA 2: Estructuras de datos eficientes para el frente de Pareto ---
        # Usamos una lista para las combinaciones y un array de NumPy para sus valores.
        self.pareto_solutions = []
        self.pareto_values = np.empty((0, 3))

    def _evaluate_solution(self, supply_indices):
        """
        Calcula los 3 objetivos para una solución de forma unificada y eficiente.
        """
        # Submatriz de distancias para los puntos de suministro seleccionados
        sub_matrix = self.dist_matrix[:, supply_indices]

        # F1: Maximizar la mínima distancia cubierta (vectorizado)
        # Lo negamos para tratar todos los objetivos como minimización
        f1_value = -np.max(np.min(sub_matrix, axis=1))

        # Asignación de puntos de demanda al suministro más cercano
        assignment_indices = np.argmin(sub_matrix, axis=1)

        # F2 y F3: Conteo de asignaciones usando bincount (muy rápido)
        counts = np.bincount(assignment_indices, minlength=self.k)
        
        f2_value = np.max(counts)
        f3_value = f2_value - np.min(counts)

        return np.array([f1_value, f2_value, f3_value])

    def _update_pareto_front(self, solution, new_values):
        """
        Actualiza el frente de Pareto usando operaciones vectorizadas de NumPy.
        Asume que todos los objetivos en `new_values` son de minimización.
        """
        # 1. ¿La nueva solución es dominada por alguna del frente actual?
        #    Si alguna solución existente es mejor o igual en todos los objetivos, la nueva es dominada.
        if np.any(np.all(self.pareto_values <= new_values, axis=1)):
            return  # La nueva solución es dominada, no hacer nada

        # 2. ¿Qué soluciones existentes son dominadas por la nueva?
        #    Identificar las soluciones existentes que son peores o iguales en todos los objetivos.
        dominated_by_new_mask = np.all(new_values <= self.pareto_values, axis=1)

        # 3. Filtrar para mantener solo las soluciones no dominadas
        self.pareto_values = self.pareto_values[~dominated_by_new_mask]
        
        # Actualizar la lista de soluciones de forma eficiente
        self.pareto_solutions = [
            s for s, is_dominated in zip(self.pareto_solutions, dominated_by_new_mask) if not is_dominated
        ]

        # 4. Añadir la nueva solución no dominada al frente
        self.pareto_values = np.vstack([self.pareto_values, new_values])
        self.pareto_solutions.append(solution)
        
    def run(self):
        """
        Ejecuta la búsqueda exhaustiva optimizada.
        """
        print("Iniciando búsqueda exhaustiva OPTIMIZADA... 🚀")
        
        potential_supplies = list(range(self.m))
        num_combinations = comb(self.m, self.k, exact=True)
        print(f"Se evaluarán {num_combinations} combinaciones. ¡Esto puede tardar!")

        all_combinations = combinations(potential_supplies, self.k)
        
        pbar = tqdm(all_combinations, total=num_combinations, desc="Procesando Combinaciones", unit=" comb")
        
        for current_solution in pbar:
            solution_list = list(current_solution)
            
            # Calcular los tres objetivos de una vez
            objective_values = self._evaluate_solution(solution_list)
            
            # Actualizar el frente de Pareto
            self._update_pareto_front(solution_list, objective_values)
            
            pbar.set_postfix(pareto_count=len(self.pareto_solutions))

        # --- MEJORA 3: Post-procesamiento y guardado único ---
        print(f"\nBúsqueda completada. Se encontraron {len(self.pareto_solutions)} soluciones no dominadas. ✅")
        
        # Revertir el signo de f1 para mostrar el valor original (maximización)
        final_values = self.pareto_values.copy()
        final_values[:, 0] *= -1 

        # Crear el DataFrame final de una sola vez
        df_final = pd.DataFrame({
            'solution': [str(sorted(s)) for s in self.pareto_solutions],
            'f1': final_values[:, 0],
            'f2': final_values[:, 1],
            'f3': final_values[:, 2]
        })

        # Guardar en CSV UNA SOLA VEZ al final del proceso
        df_final.to_csv(self.route_solutions, index=False)
        print(f"Los resultados se han guardado en '{self.route_solutions}'.")
        print("\nDataFrame final de soluciones de Pareto:")
        print(df_final)

In [ ]:
import pandas as pd
import numpy as np
import os
from itertools import combinations
from scipy.special import comb
from tqdm import tqdm
import time
from multiprocessing import Pool, cpu_count
from functools import partial

# --- MEJORA 4: Compilación JIT con Numba ---
# Importamos Numba. Si no lo tienes, instálalo: pip install numba
from numba import njit

# -----------------------------------------------------------------------------
# --- Funciones puras preparadas para Numba y Paralelización ---
# -----------------------------------------------------------------------------
# Movemos la lógica de cálculo fuera de la clase para que sea más fácil 
# de compilar con Numba y de usar con multiprocessing.

@njit
def evaluate_solution_numba(dist_matrix, supply_indices, k):
    """
    Función de evaluación compilada con Numba.
    Es una versión "pura": solo depende de sus argumentos de entrada.
    """
    # El casting a np.array es importante para que Numba infiera los tipos
    supply_indices = np.array(supply_indices) 
    sub_matrix = dist_matrix[:, supply_indices]

    # F1: Maximizar la mínima distancia (negado para minimizar)
    min_distances_per_demand = np.empty(sub_matrix.shape[0])
    for i in range(sub_matrix.shape[0]):
        min_distances_per_demand[i] = np.min(sub_matrix[i, :])
    f1_value = -np.max(min_distances_per_demand)

    # Asignación y F2/F3
    assignment_indices = np.empty(sub_matrix.shape[0], dtype=np.int64)
    for i in range(sub_matrix.shape[0]):
        assignment_indices[i] = np.argmin(sub_matrix[i, :])
    
    # np.bincount es soportado por Numba
    counts = np.bincount(assignment_indices, minlength=k)
    f2_value = np.max(counts)
    f3_value = f2_value - np.min(counts)

    return np.array([f1_value, f2_value, f3_value])

@njit
def update_pareto_front_numba(pareto_values, new_values):
    """
    Actualiza el frente de Pareto (solo los valores) de forma compilada.
    Devuelve la máscara de los elementos a eliminar y si se debe añadir el nuevo.
    """
    # 1. ¿La nueva solución es dominada?
    if np.any(np.all(pareto_values <= new_values, axis=1)):
        return np.zeros(pareto_values.shape[0], dtype=np.bool_), False

    # 2. ¿Qué soluciones existentes son dominadas por la nueva?
    dominated_mask = np.all(new_values <= pareto_values, axis=1)

    return dominated_mask, True

# -----------------------------------------------------------------------------
# --- Clase Principal Optimizada ---
# -----------------------------------------------------------------------------

class BusquedaParalelaNumba:
    def __init__(self, archive, k, m):
        folder_distances = "./data/distances/demand/"
        route_distances = os.path.join(folder_distances, f"{archive}.csv")
        df_distances_demand = pd.read_csv(route_distances, index_col=0)

        self.dist_matrix = df_distances_demand.to_numpy()
        self.supply_point_names = df_distances_demand.columns.to_numpy()
        self.k = k
        self.m = m
        self.route_solutions = f"{archive}_optimized.csv" # Guardar en otro archivo

        if self.m != self.dist_matrix.shape[1]:
            raise ValueError(f"Conflicto de dimensiones: m={self.m}, columnas={self.dist_matrix.shape[1]}")

        self.pareto_solutions = []
        self.pareto_values = np.empty((0, 3))

    def _update_pareto_front_instance(self, solution, new_values):
        """
        Método de la instancia que utiliza la función Numba para actualizar
        tanto los valores como la lista de soluciones.
        """
        if self.pareto_values.shape[0] == 0:
            self.pareto_values = np.array([new_values])
            self.pareto_solutions.append(solution)
            return

        dominated_mask, add_new = update_pareto_front_numba(self.pareto_values, new_values)
        
        if not add_new:
            return

        # Filtrar soluciones y valores no dominados
        keep_mask = ~dominated_mask
        self.pareto_values = self.pareto_values[keep_mask]
        self.pareto_solutions = [s for i, s in enumerate(self.pareto_solutions) if keep_mask[i]]

        # Añadir la nueva solución
        self.pareto_values = np.vstack([self.pareto_values, new_values])
        self.pareto_solutions.append(solution)

    def process_chunk(self, chunk_of_combinations):
        """
        Función que será ejecutada por cada proceso hijo.
        Procesa un trozo de las combinaciones y devuelve su frente de Pareto local.
        """
        local_pareto_solutions = []
        local_pareto_values = np.empty((0, 3))

        for solution_tuple in chunk_of_combinations:
            # Reutilizamos la lógica de Numba para actualizar el frente local
            objective_values = evaluate_solution_numba(self.dist_matrix, solution_tuple, self.k)
            
            # Actualiza el frente de Pareto LOCAL
            dominated_mask, add_new = update_pareto_front_numba(local_pareto_values, objective_values)
            if add_new:
                keep_mask = ~dominated_mask
                local_pareto_values = local_pareto_values[keep_mask]
                local_pareto_solutions = [s for i, s in enumerate(local_pareto_solutions) if keep_mask[i]]
                local_pareto_values = np.vstack([local_pareto_values, objective_values])
                local_pareto_solutions.append(list(solution_tuple))

        return local_pareto_solutions, local_pareto_values

    def run(self):
        print("Iniciando búsqueda EXTREMADAMENTE OPTIMIZADA (Paralela + Numba)... 🏎️💨")
        
        potential_supplies = list(range(self.m))
        num_combinations = comb(self.m, self.k, exact=True)
        print(f"Se evaluarán {num_combinations} combinaciones.")

        all_combinations = combinations(potential_supplies, self.k)

        # --- MEJORA 5: Paralelización con Multiprocessing ---
        num_cores = cpu_count()
        # Ajustar el tamaño del chunk. Un buen punto de partida es dividir el total entre el número de cores.
        # chunks_size puede necesitar ajuste para un rendimiento óptimo.
        chunk_size = max(1, num_combinations // (num_cores * 4)) 
        print(f"Usando {num_cores} núcleos con un tamaño de chunk de {chunk_size}.")

        # Creamos una "piscina" de procesos trabajadores
        with Pool(processes=num_cores) as pool:
            # Dividimos el iterador de combinaciones en chunks
            # Creamos un generador de chunks para no almacenar todas las combinaciones en memoria
            def chunk_generator(iterator, size):
                chunk = []
                for item in iterator:
                    chunk.append(item)
                    if len(chunk) == size:
                        yield chunk
                        chunk = []
                if chunk:
                    yield chunk

            chunks = chunk_generator(all_combinations, chunk_size)
            num_chunks = (num_combinations + chunk_size - 1) // chunk_size # Ceil division

            # Ejecutamos `process_chunk` en paralelo para cada chunk
            # `tqdm` nos muestra el progreso de los chunks procesados
            results = list(tqdm(pool.imap(self.process_chunk, chunks), total=num_chunks, desc="Procesando Chunks", unit=" chunk"))

        print("\nFase de procesamiento paralelo completada. Fusionando resultados...")

        # --- Fusión de los frentes de Pareto locales ---
        for local_solutions, local_values in tqdm(results, desc="Fusionando Frentes de Pareto"):
            for i, solution in enumerate(local_solutions):
                self._update_pareto_front_instance(solution, local_values[i])
        
        print(f"\nBúsqueda completada. Se encontraron {len(self.pareto_solutions)} soluciones no dominadas. ✅")
        
        self.save_results()

    def save_results(self):
        if not self.pareto_solutions:
            print("No se encontraron soluciones de Pareto.")
            return

        final_values = self.pareto_values.copy()
        final_values[:, 0] *= -1 

        df_final = pd.DataFrame({
            'solution': [str(sorted(s)) for s in self.pareto_solutions],
            'f1': final_values[:, 0],
            'f2': final_values[:, 1],
            'f3': final_values[:, 2]
        })

        df_final.to_csv(self.route_solutions, index=False)
        print(f"Los resultados se han guardado en '{self.route_solutions}'.")
        print("\nDataFrame final de soluciones de Pareto:")
        print(df_final)

# --- Punto de entrada principal ---
# Es CRUCIAL usar `if __name__ == '__main__':` cuando se usa multiprocessing
if __name__ == '__main__':
    # --- Parámetros de ejemplo ---
    # CUIDADO: La búsqueda exhaustiva crece exponencialmente.
    # m=20, k=10 ya es un número gigantesco (184,756 combinaciones)
    # m=30, k=10 es enorme (30,045,015)
    # m=50, k=5 es factible (2,118,760)
    
    M_TOTAL = 20  # Número total de puntos de suministro
    K_SELECT = 6  # Número de puntos a seleccionar
    
    # Simular un archivo de datos para que el ejemplo sea ejecutable
    if not os.path.exists("./data/distances/demand/"):
        os.makedirs("./data/distances/demand/")
    
    num_demand_points = 100
    dummy_distances = np.random.rand(num_demand_points, M_TOTAL) * 100
    dummy_df = pd.DataFrame(dummy_distances, columns=[f'Supply_{i}' for i in range(M_TOTAL)])
    dummy_df.to_csv("./data/distances/demand/dummy_data.csv")
    
    
    start_time = time.time()
    
    # Instanciar y ejecutar la búsqueda
    exhaustive_search = BusquedaParalelaNumba(archive="dummy_data", k=K_SELECT, m=M_TOTAL)
    exhaustive_search.run()
    
    end_time = time.time()
    print(f"\nTiempo total de ejecución: {end_time - start_time:.2f} segundos.")

# Ejecución

In [5]:
if __name__ == '__main__':
    # Asegúrate de que la carpeta de datos exista
    if not os.path.exists("./data/distances/demand/"):
        print("Error: La carpeta de datos no existe. Asegúrate de tener la estructura de directorios correcta.")
    else:
        # Aquí debes poner el nombre de tu archivo (sin .csv), k y m correctos
        # ¡CUIDADO! Valores grandes de k y m harán que el programa tarde muchísimo.
        analisis_exhaustivo = BusquedaExhaustivaOptimizada(archive, k, m)
        analisis_exhaustivo.run()

Iniciando búsqueda exhaustiva OPTIMIZADA... 🚀
Se evaluarán 10272278170 combinaciones. ¡Esto puede tardar!


Procesando Combinaciones:   0%|          | 25713/10272278170 [00:18<2103:29:21, 1356.51 comb/s, pareto_count=13]


KeyboardInterrupt: 

# Pasar el csv a txt para comparativa de soluciones

In [5]:
def guardar_pareto_txt(ruta_csv, ruta_txt):
  """
  Lee un archivo CSV con soluciones de Pareto y guarda los datos de las
  columnas 'f1', 'f2' y 'f30' en un archivo de texto con un formato específico.

  Args:
    ruta_csv (str): La ruta al archivo CSV de entrada.
    ruta_txt (str): La ruta al archivo de texto de salida.
  """
  # Lee el archivo CSV en un DataFrame de pandas
  df = pd.read_csv(ruta_csv)

  # Abre el archivo de texto en modo de escritura
  with open(ruta_txt, 'w') as f:
    # Itera sobre cada fila del DataFrame
    for _, fila in df.iterrows():
      # Extrae y redondea los valores de las columnas
      f1 = fila['f1']
      f2 = fila['f2']
      f3 = fila['f3'] # Se ajusta el valor de f30

      # Escribe los valores formateados en el archivo de texto
      f.write(f"{f1}\t{f2}\t{f3}\n")

# Ejemplo de uso de la función
guardar_pareto_txt('./Pareto_front/WorkSpace 1000_50_5.csv', './Pareto_front/WorkSpace 1000_50_5.txt')

print("¡El archivo 'pareto_formateado.txt' ha sido creado con éxito!")

¡El archivo 'pareto_formateado.txt' ha sido creado con éxito!


# Soluciones exhaustivas vs frente de pareto paper previo

In [6]:
# Carga
ruta_solution="./Pareto_front/WorkSpace 1000_50_5.csv"
ruta_pareto="./Pareto_front_paper/"+archive+".txt"
df_csv_full, df_csv_obj, df_txt = load_solutions(ruta_solution, ruta_pareto)

In [7]:
# Unir y encontrar frente de Pareto
all_solutions = np.vstack([df_csv_obj.values, df_txt.values])
pareto_mask = get_pareto_front(all_solutions)
pareto_solutions = all_solutions[pareto_mask]

# Arrays para detección de comunes
csv_array = df_csv_obj.values
txt_array = df_txt.values

# Clasificar
colors_csv = classify_solutions(df_csv_obj, pareto_solutions, txt_array)
colors_txt = classify_solutions(df_txt, pareto_solutions, csv_array)

# Mostrar
html_csv = get_styled_table(df_csv_full, colors_csv)
html_txt = get_styled_table(df_txt, colors_txt)

# Mostrar lado a lado con HTML
display(HTML(f"""
<div style="display: flex; gap: 30px;">
  <div style="flex: 1;">
    <h3>Soluciones CSV (con ID)</h3>
    {html_csv}
  </div>
  <div style="flex: 1;">
    <h3>Soluciones TXT</h3>
    {html_txt}
  </div>
</div>
"""))


,solution,f1,f2,f3
0,"[1, 4, 14, 27, 29]",379.969736,259,160
1,"[2, 8, 11, 27, 29]",481.133038,203,6
2,"[2, 17, 19, 23, 29]",458.802790,209,22
3,"[3, 7, 12, 27, 29]",442.764046,229,104
4,"[3, 9, 12, 27, 29]",442.764046,233,103
5,"[3, 9, 13, 27, 29]",433.416659,243,113
6,"[3, 10, 15, 27, 30]",441.834811,241,125
7,"[3, 12, 15, 27, 30]",450.699456,227,83
8,"[4, 14, 27, 29, 46]",377.005305,281,210
9,"[4, 14, 27, 29, 47]",377.005305,284,202


## Truncando los decimales para comparar correctamente

In [8]:
df_csv_full['f1'] = df_csv_full['f1'].astype(int)
df_csv_obj['f1'] = df_csv_obj['f1'].astype(int)

In [9]:
# Unir y encontrar frente de Pareto
all_solutions = np.vstack([df_csv_obj.values, df_txt.values])
pareto_mask = get_pareto_front(all_solutions)
pareto_solutions = all_solutions[pareto_mask]

# Arrays para detección de comunes
csv_array = df_csv_obj.values
txt_array = df_txt.values

# Clasificar
colors_csv = classify_solutions(df_csv_obj, pareto_solutions, txt_array)
colors_txt = classify_solutions(df_txt, pareto_solutions, csv_array)

# Mostrar
html_csv = get_styled_table(df_csv_full, colors_csv)
html_txt = get_styled_table(df_txt, colors_txt)

# Mostrar lado a lado con HTML
display(HTML(f"""
<div style="display: flex; gap: 30px;">
  <div style="flex: 1;">
    <h3>Soluciones CSV (con ID)</h3>
    {html_csv}
  </div>
  <div style="flex: 1;">
    <h3>Soluciones TXT</h3>
    {html_txt}
  </div>
</div>
"""))


,solution,f1,f2,f3
0,"[1, 4, 14, 27, 29]",379,259,160
1,"[2, 8, 11, 27, 29]",481,203,6
2,"[2, 17, 19, 23, 29]",458,209,22
3,"[3, 7, 12, 27, 29]",442,229,104
4,"[3, 9, 12, 27, 29]",442,233,103
5,"[3, 9, 13, 27, 29]",433,243,113
6,"[3, 10, 15, 27, 30]",441,241,125
7,"[3, 12, 15, 27, 30]",450,227,83
8,"[4, 14, 27, 29, 46]",377,281,210
9,"[4, 14, 27, 29, 47]",377,284,202


# Soluciones extraidas vs soluciones exactas

In [10]:
# Carga
ruta_solution="Solutions/Multiprocessing/"+archive+"/"+archive+".csv"
ruta_pareto="./Pareto_front/WorkSpace 1000_50_5.txt"
df_csv_full, df_csv_obj, df_txt = load_solutions(ruta_solution, ruta_pareto)

In [11]:
# Unir y encontrar frente de Pareto
all_solutions = np.vstack([df_csv_obj.values, df_txt.values])
pareto_mask = get_pareto_front(all_solutions)
pareto_solutions = all_solutions[pareto_mask]

# Arrays para detección de comunes
csv_array = df_csv_obj.values
txt_array = df_txt.values

# Clasificar
colors_csv = classify_solutions(df_csv_obj, pareto_solutions, txt_array)
colors_txt = classify_solutions(df_txt, pareto_solutions, csv_array)

# Mostrar
html_csv = get_styled_table(df_csv_full, colors_csv)
html_txt = get_styled_table(df_txt, colors_txt)

# Mostrar lado a lado con HTML
display(HTML(f"""
<div style="display: flex; gap: 30px;">
  <div style="flex: 1;">
    <h3>Soluciones CSV (con ID)</h3>
    {html_csv}
  </div>
  <div style="flex: 1;">
    <h3>Soluciones TXT</h3>
    {html_txt}
  </div>
</div>
"""))


,solution,f1,f2,f3
0,"[13, 23, 29, 45, 49]",412.506970,247,148
1,"[2, 8, 11, 27, 29]",481.133038,203,6
2,"[1, 4, 14, 27, 29]",379.969736,259,160
3,"[4, 14, 27, 29, 46]",377.005305,281,210
4,"[3, 12, 15, 27, 30]",450.699456,227,83
5,"[3, 9, 13, 27, 29]",433.416659,243,113
6,"[3, 10, 15, 27, 30]",441.834811,241,125
7,"[13, 20, 23, 29, 49]",415.120464,233,129
8,"[2, 17, 19, 23, 29]",458.802790,209,22
9,"[3, 9, 12, 27, 29]",442.764046,233,103
